In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
##from tensorflow.keras.callbacks import Mode
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as k
from tensorflow.keras.utils import *
import pickle
import tensorflow as tf
import re
print(tf.__version__)

2.8.2


In [8]:
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as K
from tensorflow.keras.utils import *
import pickle
import tensorflow as tf
import re
print(tf.__version__)

2.8.2


In [9]:
class Simple_Relu(object):
  def __init__(self,hidden_layers,hidden_nodes,input_shape,output_shape):
    self.hidden_layers=hidden_layers
    self.hidden_nodes=hidden_nodes
    self.input_shape=input_shape
    self.output_shape=output_shape
    self.initialize()
  def initialize(self):
    inputs=keras.Input(shape=(self.input_shape,))
    X_in=Dense(units =self.hidden_nodes, activation = 'relu', name = "layer_relu_1")(inputs)
    for i in range(0,self.hidden_layers-2):
      X_in=Dense(units =self.hidden_nodes, activation = 'relu', name = "layer_relu"+str(i+2))(X_in)
      
    output_layer=Dense(units =self.output_shape, activation = "softmax", name = "output"+str(self.hidden_layers))(X_in)
    self.model=keras.Model(inputs=inputs,outputs=output_layer,name="simple_relu")



In [10]:
eps=0.1
beta=4

In [11]:
class softrelu(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
        super(softrelu, self).__init__(**kwargs)
  def build(self, input_shape):
        super(softrelu, self).build(input_shape)  # Be sure to call this at the end
  def call(self, x):
    activation = tf.keras.backend.sigmoid(beta*x)
    return activation

  ##def compute_output_shape(self, input_shape):
    ## return input_shape



In [24]:
from keras.utils.generic_utils import get_custom_objects
def soft_relu(x):
    return tf.keras.backend.sigmoid(beta*x)

decoupled learning

In [25]:
class double_network(object):
  def __init__(self,hidden_layers,hidden_nodes,input_shape,output_shape):
    self.hidden_layers=hidden_layers

    self.hidden_nodes=hidden_nodes

    self.input_shape=input_shape

    self.output_shape=output_shape

    self.initialize()

  def initialize(self):
    inputs=keras.Input(shape=(self.input_shape,))
    
    X_feature=Dense(units =self.hidden_nodes,activation='linear',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',
                    name = "feature_network_L_1")(inputs)

    feature_input=tf.keras.layers.Activation('relu')(X_feature)
    get_custom_objects().update({'custom_activation_soft': tf.keras.layers.Activation(soft_relu)})
    soft_activation=tf.keras.layers.Activation(soft_relu, name='SpecialActivation_soft')(X_feature)

    X_value=Dense(units =self.hidden_nodes,activation='linear',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',
                  name = "value_intermediate_L_1")(inputs)

                  
    value_input=tf.keras.layers.Multiply()([X_value,soft_activation])
    



    for i in range(0,self.hidden_layers-2):

      X_feature=Dense(units =self.hidden_nodes, activation = 'linear', name = "feature_network_L_"+str(i+2))(feature_input)

      feature_input=tf.keras.layers.Activation('relu')(X_feature)
      
      get_custom_objects().update({'custom_activation_soft': tf.keras.layers.Activation(soft_relu)})
      soft_activation=tf.keras.layers.Activation(soft_relu)(X_feature)
      

      X_value=Dense(units =self.hidden_nodes,activation='linear',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',
                  name = "value_intermediate_L_"+str(i+2))(value_input)

      value_input=tf.keras.layers.Multiply(name = "multiply_L_"+str(i+2))([X_value,soft_activation])
      
       


      
    output_layer=Dense(units =self.output_shape, activation = "softmax", name = "layer_relu"+str(self.hidden_layers))(value_input)

    self.model=keras.Model(inputs=inputs,outputs=output_layer,name="gated_decopupled_model")

    
  


FLNPF 

In [13]:
from keras.utils.generic_utils import get_custom_objects
def custom_activation(x):
    return tf.keras.backend.sign(tf.keras.backend.relu(x))

In [17]:
class FLNPF_II(object):
  def __init__(self,hidden_layers,hidden_nodes,input_shape,output_shape):
    self.hidden_layers=hidden_layers
    self.hidden_nodes=hidden_nodes
    self.input_shape=input_shape
    self.output_shape=output_shape
    self.initialize()
  def initialize(self):
    inputs=keras.Input(shape=(self.input_shape,))
    
    X_feature=Dense(units =self.hidden_nodes,activation='relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',
                    name = "feature_network_L_1")(inputs)

    X_value=Dense(units =self.hidden_nodes,activation='linear',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',
                  name = "value_intermediate_L_1")(inputs)
    get_custom_objects().update({'custom_activation': tf.keras.layers.Activation(custom_activation)})
    hard_activation=tf.keras.layers.Activation(custom_activation, name='SpecialActivation')(X_feature)

    value_input=tf.keras.layers.Multiply()([X_value,hard_activation])
    



    for i in range(0,self.hidden_layers-2):

      X_feature=Dense(units =self.hidden_nodes, activation = 'relu', name = "feature_network_L_"+str(i+2))(X_feature)

      X_value=Dense(units =self.hidden_nodes,activation='linear',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',
                  name = "value_intermediate_L_"+str(i+2))(value_input)
      get_custom_objects().update({'custom_activation': tf.keras.layers.Activation(custom_activation)})
      hard_activation=tf.keras.layers.Activation(custom_activation)(X_feature)

      value_input=tf.keras.layers.Multiply()([X_value,hard_activation])
      
       


      
    output_layer=Dense(units =self.output_shape, activation = "softmax", name = "VALUE_NETWORK_OUTPUT"+str(self.hidden_layers))(value_input)

    self.model=keras.Model(inputs=inputs,outputs=output_layer,name="fixed_learning_npf_II_model")
    

In [26]:
def load_data():
  ((trainX, trainY), (testX, testY)) = mnist.load_data()
  trainX = trainX.reshape((trainX.shape[0], 28 * 28 * 1))
  testX = testX.reshape((testX.shape[0], 28 * 28 * 1))
  trainX = trainX.astype("float32") / 255.0
  testX = testX.astype("float32") / 255.0
  lb = LabelBinarizer()
  trainY = lb.fit_transform(trainY)
  testY = lb.transform(testY)
  return trainX ,trainY,testX,testY

decoupled_learning

In [27]:
trainX,trainY,testX,testY=load_data()
model=double_network(6,128,784,10)
layer=model.model.layers
model.model.summary()
print(model.model.layers)
model.model.compile(
optimizer=tf.keras.optimizers.Adam(0.0003),  # Optimizer
loss="categorical_crossentropy", metrics="accuracy")
model.model.fit(trainX,trainY,epochs=5,batch_size=32,validation_data=(testX,testY))

Model: "gated_decopupled_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 784)]        0           []                               
                                                                                                  
 feature_network_L_1 (Dense)    (None, 128)          100480      ['input_6[0][0]']                
                                                                                                  
 value_intermediate_L_1 (Dense)  (None, 128)         100480      ['input_6[0][0]']                
                                                                                                  
 SpecialActivation_soft (Activa  (None, 128)         0           ['feature_network_L_1[0][0]']    
 tion)                                                                       

FLNPF_INDEPENDENT_INITIALIZATION

In [ ]:
import tensorflow as tf
if __name__=="__main__":
  trainX,trainY,testX,testY=load_data()
  model=FLNPF_II(6,128,784,10)
  layer=model.model.layers
  layers=[]
  for l in layer:
    if l.name[0]=='f':
      l.trainable=False
  model.model.summary()
  print(model.model.layers)
  model.model.compile(
  optimizer=tf.keras.optimizers.Adam(0.0003),  # Optimizer
  loss="categorical_crossentropy", metrics="accuracy")
  model.model.fit(trainX,trainY,epochs=5,batch_size=32,validation_data=(testX,testY))


flnpf_dependent_initialization

In [ ]:
trainX,trainY,testX,testY=load_data()
model=FLNPF_II(6,128,784,10)
layer=model.model.layers
layers_value=[]
layers_feature=[]
for l in layer:
  if l.name[0]=='f':
    layers_feature.append(l)
  elif l.name[0]=='v':
    layers_value.append(l)
for l_1 in layer:
  if l_1.name[0]=='f':
    l_1.trainable=False
model.model.compile(
optimizer=tf.keras.optimizers.Adam(0.0003),  # Optimizer
loss="categorical_crossentropy", metrics="accuracy")
for l1,l2 in zip(layers_value,layers_feature):
  l1.set_weights(l2.get_weights())
model.model.summary()
print(model.model.layers)
model.model.fit(trainX,trainY,epochs=5,batch_size=32,validation_data=(testX,testY))

simple FLNPF

In [29]:
trainX,trainY,testX,testY=load_data()
model=Simple_Relu(6,128,784,10)
model.model.compile(
optimizer=tf.keras.optimizers.Adam(0.0003),  # Optimizer
loss="categorical_crossentropy", metrics="accuracy")
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.model.fit(trainX,trainY,epochs=5,batch_size=32,validation_data=(testX,testY),callbacks=callbacks_list)
model.model.load_weights("weights.best.hdf5")
layer_relu=model.model.layers
model_npf=FLNPF_II(6,128,784,10)
layer_npf=model_npf.model.layers

layers_relu=[]
layers_feature=[]
for l in layer_relu:
  if l.name[0]=='l':
    layers_relu.append(l)
for l in layer_npf :
  if l.name[0]=='f':
    layers_feature.append(l)
for l in layer_npf:
  if l.name[0]=='f':
    l.trainable=False
for l1,l2 in zip(layers_relu,layers_feature):
  l2.set_weights(l1.get_weights())
model_npf.model.compile(
optimizer=tf.keras.optimizers.Adam(0.0003),  # Optimizer
loss="categorical_crossentropy", metrics="accuracy")
model.model.summary()
print(model.model.layers)
model_npf.model.fit(trainX,trainY,epochs=5,batch_size=32,validation_data=(testX,testY))

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.3100 - accuracy: 0.9069 - val_loss: 0.1393 - val_accuracy: 0.9569
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1192 - accuracy: 0.9638 - val_loss: 0.1077 - val_accuracy: 0.9663
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0860 - accuracy: 0.9739 - val_loss: 0.1028 - val_accuracy: 0.9676
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0660 - accuracy: 0.9793 - val_loss: 0.1007 - val_accuracy: 0.9698
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0513 - accuracy: 0.9839 - val_loss: 0.0877 - val_accuracy: 0.9750
Model: "simple_relu"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 784)]             0         
                                                                 
